In [ ]:
using JuMP
using Gurobi
using LinearAlgebra
using Distributions
using Random
using PDMats
using MLJ
using Tables
using DataFrames

Random.seed!(42)

In [ ]:
using Revise

includet("../models/forward.jl")
import .Forward as Forward

includet("../models/inversedemand.jl")
import .InverseDemand as IODemand

includet("../models/inverselinreg.jl")
import .InverseLinReg as IOLinReg

includet("../datagen/data-generation.jl")
import .DataGeneration as DataGen


In [ ]:
Linear = @load LinearRegressor pkg = "MLJLinearModels"

# Data generation

Steps for tomorrow:
1. Save two datasets
4. Split each dataset into train and test sets
5. Run DFL-IO with each training set, get weights $\theta^{\text{IO,close}}$ and $\theta^{\text{IO,wide}}$
6. Run linear regression on training set, get weights $\theta^{\text{R,close}}$ and $\theta^{\text{R,wide}}$
7. Compare all models on each training set, for each model:
    1. Predict demand using weights and features
    2. Solve MCFND with demand, get design variables $\hat{y}$
    3. Solve MCF-Flow with fixed design variables $\hat{y}$, get $\hat{x}$
    4. Compare costs of $\hat{y}, \hat{x}$ with optimal $x^*, y^*$ 

## Problem parameters

Make a smaller problem with 1 commodity and 2 possible arcs, one low-ish capacity ($C$) and one high ($\infty$)


In [ ]:
forward_params = Forward.Params(
    n_paths=2, 
    n_commodities=1,
    capacities=[100, 100000],
    design_costs=[10, 100],
    flow_costs=[10 100]',
    enabled_flows=ones(Bool, (2, 1))
)

datagen_params = DataGen.DataGenParams(
    weights=[1.5 -3 2], 
    noise_variance=[5.0^2]
)

inverse_params = IOLinReg.Params(
    n_features=datagen_params.n_features, 
    forward_params=forward_params, 
    with_noise=true
)

## Dataset creation

Generate two datasets using fixed weights $\Theta$:
- $\mathcal{D}_{\text{close}}$ with $\mathbb{E}[d] = C$ 
- $\mathcal{D}_{\text{wide}}$ with $\mathbb{E}[d] \ll C$. 

Procedure for each data point in the dataset:
1. Draw $\phi_1, \ldots, \phi_{m-1} \sim U(a, b)$ for some $a, b$
2. Set $\phi_m$ such that $\sum_{i=1}^m \theta_i \phi_i = \mathbb{E}[d]$
3. Draw noise $\epsilon \sim \mathcal{N}(0, \sigma^2)$ and compute $d = \sum_{i=1}^m \theta_i \phi_i + \epsilon$
4. Solve MCFND for $d$ 
5. Datapoint $(\phi, d, x^*, y^*) \in \mathcal{D}$


In [ ]:
n_points = 100
gurobi_env = Gurobi.Env()

close_target_demand = 100
wide_target_demand = 20

close_dataset = DataGen.generate_dataset(forward_params, datagen_params, n_points=n_points, target_demand=close_target_demand, gurobi_env=gurobi_env)
wide_dataset = DataGen.generate_dataset(forward_params, datagen_params, n_points=n_points, target_demand=wide_target_demand, gurobi_env=gurobi_env)

show(close_dataset[1])
println()
show(wide_dataset[1])

Split each dataset into training and testing

In [ ]:
test_train_split = 0.8

close_train_dataset, close_test_dataset = partition(close_dataset, test_train_split)
wide_train_dataset, wide_test_dataset = partition(wide_dataset, test_train_split)

## Model training

### Inverse-DFL model

In [ ]:
function train_inverse_model(training_dataset)
    model = IOLinReg.create_problem(inverse_params, training_dataset, gurobi_env=gurobi_env)

    return IOLinReg.solve_problem!(model, inverse_params)
end

close_inverse_solution = train_inverse_model(close_train_dataset)
wide_inverse_solution = train_inverse_model(wide_train_dataset)

In [ ]:
println("IO model close weights: $(close_inverse_solution.weights)")
println("IO model wide weights: $(wide_inverse_solution.weights)")

### Linear regression model

In [ ]:
function convert_dataset_to_mlj(dataset)
    features = DataFrame(vcat(map(sol -> sol.linreg_features', dataset)...), :auto)
    demands = vcat(map(sol -> sol.actual_demands, dataset)...)

    return features, demands
end

In [ ]:
function train_linear_model(training_dataset)
    features, demands = convert_dataset_to_mlj(training_dataset)

    model = Linear()
    mach = machine(model, features, demands)
    fit!(mach)

    return mach
end

close_linreg_machine = train_linear_model(close_train_dataset)
wide_linreg_machine = train_linear_model(wide_train_dataset)

## Model evaluation

### Making predictions on the test set

Using the inverse model

In [ ]:
function predict_inverse_model(inverse_solution, test_dataset)
    features = map(row -> row.linreg_features, test_dataset)
    predict = f -> IOLinReg.predict_inverse_model(inverse_solution, f)

    return vcat(map(predict, features)...)
end

close_io_pred_demands = predict_inverse_model(close_inverse_solution, close_test_dataset)
wide_io_pred_demands = predict_inverse_model(wide_inverse_solution, wide_test_dataset)

Using the linear regression model

In [ ]:
function predict_linear_model(linreg_machine, test_dataset)
    features, _ = convert_dataset_to_mlj(test_dataset)
    return predict(linreg_machine, features)
end

close_linreg_pred_demands = predict_linear_model(close_linreg_machine, close_test_dataset)
wide_linreg_pred_demands = predict_linear_model(wide_linreg_machine, wide_test_dataset)

## Evaluation 